In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Tuberous_Sclerosis/GSE14334'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptomic analysis of human lung development"
!Series_summary	"We decompose the genome-wide expression patterns in 38 embryonic human lung (53-154 days post conception/dpc) into their independent, dominant directions of transcriptomic sample variation in order togain global insight of the developing human lung transcriptome.The characteristic genes and their corresponding bio–ontologic attribute profile for the latter were identified. We noted the over–representation of lung specific attributes (e.g., surfactant proteins) traditionally associated with later developmental stages, and highly ranked attributes (e.g., chemokine–immunologic processes) not previously reported nor immediately apparent in an early lung development context. We defined the 3,223–gene union of the characteristic genes of the 3 most dominant sources of variation as the developing lung characteristic sub–transcriptome (DLCS). It may be regarded as the minimal gene set des

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check gene expression data availability
is_gene_available = True  # Based on background information summary

# Identify variable availability
trait_row = 3  # The key where 'Disease Status' is recorded
age_row = None  # No clear reference to age
gender_row = 1  # The key where 'Sex' is recorded

# Define conversion functions
def convert_trait(value):
    try:
        trait_value = value.split(':')[-1].strip()
        return 1 if trait_value == 'Tuberous Sclerosis' else 0
    except Exception:
        return None

def convert_age(value):
    return None  # Not applicable as age data is unavailable

def convert_gender(value):
    try:
        gender_value = value.split(':')[-1].strip()
        if gender_value == 'Male':
            return 1
        elif gender_value == 'Female':
            return 0
        else:
            return None
    except Exception:
        return None

# Save cohort information
save_cohort_info('GSE14334', './preprocessed/Tuberous_Sclerosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Tuberous_Sclerosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Tuberous_Sclerosis/trait_data/GSE14334.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM358631': [0, None], 'GSM358632': [0, None], 'GSM358633': [0, None], 'GSM358634': [0, 0], 'GSM358635': [0, None], 'GSM358636': [0, 0], 'GSM358637': [0, None], 'GSM358638': [0, 1], 'GSM358639': [0, 1], 'GSM358640': [0, 1], 'GSM358641': [0, 1], 'GSM358642': [1, 1], 'GSM358643': [0, 1], 'GSM358644': [0, 0], 'GSM358645': [0, 1], 'GSM358646': [0, 1], 'GSM358647': [0, 1], 'GSM358648': [0, 1], 'GSM358649': [0, 0], 'GSM358650': [0, 1], 'GSM358651': [0, 1], 'GSM358652': [0, None], 'GSM358653': [0, 1], 'GSM358654': [0, 1], 'GSM358655': [0, 1], 'GSM358656': [0, 0], 'GSM358657': [0, 0], 'GSM358658': [0, 0], 'GSM358659': [0, 1], 'GSM358660': [0, 1], 'GSM358661': [0, None], 'GSM358662': [0, 0], 'GSM358663': [0, 1], 'GSM358664': [0, 0], 'GSM358665': [0, 1], 'GSM358666': [0, None], 'GSM358667': [0, None], 'GSM358668': [0, None]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify keys for identifier and gene symbol
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and obtain the resulting gene expression dataframe named "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Tuberous_Sclerosis/gene_data/GSE14334.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Tuberous_Sclerosis')

# 4. Save the cohort information.
save_cohort_info('GSE14334', './preprocessed/Tuberous_Sclerosis/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Tuberous_Sclerosis/GSE14334.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Tuberous_Sclerosis', the least common label is '1.0' with 1 occurrences. This represents 3.57% of the dataset.
The distribution of the feature 'Tuberous_Sclerosis' in this dataset is severely biased.

For the feature 'Gender', the least common label is '0.0' with 9 occurrences. This represents 32.14% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Tuberous_Sclerosis/cohort_info.json
